<a href="https://colab.research.google.com/github/taraskin5201/Hospital/blob/main/Hospital.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Завдання


Напишіть чат-бота для роботи лікарні. У чат-бота має бути 2 основних інструмента:
* доступ до векторної бази даних з документами
* доступ до реляційної бази даних(SQL)

<br>

**Векторна база даних**

Створіть векторну базу даних на *Pineconne* з [документами](https://github.com/HalyshAnton/ITStep-AI/tree/exam/data/exam/hospital). Виконайте наступні кроки
* прочитайте вміст кожного документа
* розбийте його на розділи
* добавте до метеданих поточний час
* добавте кожен розділ у векторну бд
* збережіть ID з назвами розділів та назвами файлів

<br>

**Реляційна база даних**

Створіть реляційну базу даних на [Supabase](https://supabase.com/), використайте SQL запити з практичних по базах даних цього курсу.
База даних має містити декілька таблиць та зв'язки між ними.

Створіть інструмент для агента за допомогою [SQLDatabase](https://python.langchain.com/docs/integrations/tools/sql_database/)

Створіть додаток вашого чат бота за допомогою *streamlit*

#Виконання завдання

**Векторна база даних**

Створіть векторну базу даних на *Pineconne* з [документами](https://github.com/HalyshAnton/ITStep-AI/tree/exam/data/exam/hospital). Виконайте наступні кроки
* прочитайте вміст кожного документа
* розбийте його на розділи
* добавте до метеданих поточний час
* добавте кожен розділ у векторну бд
* збережіть ID з назвами розділів та назвами файлів

###Векторна база даних

In [1]:
!pip install python-docx pypdf sentence-transformers


  Using cached pinecone_client-6.0.0-py3-none-any.whl.metadata (3.4 kB)
Using cached pinecone_client-6.0.0-py3-none-any.whl (6.7 kB)


In [2]:
!pip install pinecone


Found existing installation: pinecone-client 6.0.0
Uninstalling pinecone-client-6.0.0:
  Successfully uninstalled pinecone-client-6.0.0
Found existing installation: pinecone 8.0.0
Uninstalling pinecone-8.0.0:
  Successfully uninstalled pinecone-8.0.0
  Using cached pinecone-8.0.0-py3-none-any.whl.metadata (11 kB)
Using cached pinecone-8.0.0-py3-none-any.whl (745 kB)


In [9]:
import os
import uuid
from datetime import datetime

from docx import Document
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer

# Colab Secrets
from google.colab import userdata

import pinecone
from pinecone import Pinecone, ServerlessSpec


# Завантаження ключів (Colab Secrets)
PINECONE_API_KEY = userdata.get("PINECONE_API_KEY")
if not PINECONE_API_KEY:
    raise ValueError("PINECONE_API_KEY не знайдено у Colab Secrets")


# Ініціалізація Pinecone
index_name = "hospital-docs"
dimension = 384  # для all-MiniLM-L6-v2

pc = Pinecone(api_key=PINECONE_API_KEY)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

index = pc.Index(index_name)


# Ініціалізація моделі
model = SentenceTransformer("all-MiniLM-L6-v2")


# Парсинг DOCX
def parse_docx_full_sections(path):
    """
    Парсить DOCX і повертає список секцій:
    - section_title = заголовок розділу
    - content = весь текст розділу без поділу на чанки
    Також включає ЗМІСТ як окрему секцію.
    """
    doc = Document(path)
    sections = []

    current_title = None
    current_text = []
    toc_text = []
    toc_found = False

    for p in doc.paragraphs:
        text = p.text.strip()
        if not text:
            continue

        if not toc_found and "зміст" in text.lower():
            toc_found = True
            toc_text.append(text)
            continue
        elif toc_found and p.style.name.startswith("Heading"):
            sections.append({
                "section_title": "ЗМІСТ",
                "content": "\n".join(toc_text)
            })
            toc_text = []
            toc_found = False

        if p.style.name.startswith("Heading"):
            if current_title:
                sections.append({
                    "section_title": current_title,
                    "content": "\n".join(current_text)
                })
            current_title = text
            current_text = []
        else:
            if toc_found:
                toc_text.append(text)
            else:
                current_text.append(text)

    if current_title:
        sections.append({
            "section_title": current_title,
            "content": "\n".join(current_text)
        })

    if toc_text:
        sections.insert(0, {
            "section_title": "ЗМІСТ",
            "content": "\n".join(toc_text)
        })

    return sections


# Парсинг PDF
def parse_pdf_full_sections(path):
    reader = PdfReader(path)
    lines = []

    for page in reader.pages:
        text = page.extract_text()
        if text:
            lines.extend([line.strip() for line in text.split("\n") if line.strip()])

    sections = []
    n = len(lines)
    i = 0

    if n > 0:
        sections.append({
            "section_title": "Заголовок документа",
            "content": lines[0]
        })
        i = 1

    toc_lines = []
    while i < n and not (
        len(lines[i]) >= 2 and
        lines[i][0].isdigit() and
        lines[i][1] == '.' and
        (len(lines[i]) == 2 or lines[i][2] == ' ')
    ):
        toc_lines.append(lines[i])
        i += 1

    if toc_lines:
        sections.append({
            "section_title": "ЗМІСТ",
            "content": "\n".join(toc_lines)
        })

    current_title = None
    current_text = []

    while i < n:
        line = lines[i]
        if len(line) >= 3 and line[0].isdigit() and line[1] == '.' and line[2] == ' ':
            if current_title:
                sections.append({
                    "section_title": current_title,
                    "content": "\n".join(current_text)
                })
            current_title = line
            current_text = []
        else:
            current_text.append(line)
        i += 1

    if current_title:
        sections.append({
            "section_title": current_title,
            "content": "\n".join(current_text)
        })

    return sections


# Upload з перевіркою
def upload_sections(sections, filename):
    vectors = []
    now = datetime.utcnow().isoformat()

    for s in sections:
        text = s["content"]
        if not text.strip():
            continue

        embedding = model.encode(text)
        if len(embedding) != dimension:
            print(
                f"Пропускаємо '{s['section_title']}' — "
                f"розмір ембедінгу {len(embedding)} ≠ {dimension}"
            )
            continue

        vector_id = str(uuid.uuid4())
        vectors.append({
            "id": vector_id,
            "values": embedding.tolist(),
            "metadata": {
                "file_name": filename,
                "section_title": s["section_title"],
                "created_at": now
            }
        })

    if vectors:
        print(f"Завантажуємо {len(vectors)} векторів у Pinecone...")
        index.upsert(vectors)
    else:
        print("Немає векторів для upsert!")

    return [
        (v["id"], v["metadata"]["section_title"], v["metadata"]["file_name"])
        for v in vectors
    ]



# Основний запуск
docx_sections = parse_docx_full_sections("ForWorkers.docx")
pdf_sections = parse_pdf_full_sections("General.pdf")

docx_ids = upload_sections(docx_sections, "ForWorkers.docx")
pdf_ids = upload_sections(pdf_sections, "General.pdf")

print("Документи завантажено у Pinecone")


/tmp/ipython-input-642493174.py:187: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow().isoformat()


Завантажуємо 13 векторів у Pinecone...
Завантажуємо 7 векторів у Pinecone...
Документи завантажено у Pinecone


###Результат розподілу DOCX документа

In [10]:
print("======Розподіл DOCX документа=======")
# Отримуємо розділи
docx_sections = parse_docx_full_sections("ForWorkers.docx")

# Прінтуємо перші кілька розділів
for i, s in enumerate(docx_sections, 1):
    print(f"\n===== Розділ {i}: {s['section_title']} =====")
    print(s['content'][:])

======Розподіл DOCX документа=======

===== Розділ 1: ЗМІСТ =====
ЗМІСТ
Вступ. Загальні положення
1.1. Мета документу
1.2. Сфера застосування
1.3. Принципи трудових відносин
Прийом на роботу та адаптація
2.1. Документи для працевлаштування
2.2. Випробувальний термін
2.3. Адаптація та наставництво
Робочий час та його облік
3.1. Нормальна тривалість робочого часу
3.2. Робочі зміни та графіки
3.3. Обідні та технологічні перерви
3.4. Облік робочого часу
3.5. Неповний робочий час
3.6. Чергування
Оплата праці
4.1. Система оплати праці
4.2. Строки та порядок виплати заробітної плати
4.3. Складові заробітної плати (оклад, доплати, премії)
4.4. Надбавки за особливі умови праці
4.5. Оплата надурочних робіт, роботи у вихідні та святкові дні
4.6. Відрахування із заробітної плати
4.7. Розрахункові листи
Відпустки
5.1. Щорічна основна відпустка (порядок надання, тривалість, графік)
5.2. Щорічні додаткові відпустки
5.3. Соціальні відпустки (у зв'язку з вагітністю та пологами, по догляду за дитиною)
5

###Результат розподілу PDF документа

In [11]:
print("======Розподіл PDF документа=======")
pdf_sections = parse_pdf_full_sections("General.pdf")

# подивитися вміст розділів
for i, s in enumerate(pdf_sections, 1):
    print(f"\n===== Розділ {i}: {s['section_title']} =====")
    print(s['content'][:])

======Розподіл PDF документа=======

===== Розділ 1: Заголовок документа =====
ДОКУМЕНТАЦІЯ  МЕДИЧНОГО  ЦЕНТРУ  "ОМЕГА-МЕД"

===== Розділ 2: ЗМІСТ =====
ЗМІСТ  1.  Загальні  відомості  про  Медичний  Центр  "Омега-Мед" 1.1.  Місія  та  Візія 1.2.  Структура  організації 1.3.  Принципи  роботи  2.  Інформаційні  Системи  Лікарні  (ІСЛ) 2.1.  Електронна  Система  Обліку  Здоров'я  (ЕСОЗ  "МедВектор") 2.1.1.  Модулі  ЕСОЗ 2.1.2.  Архітектура  та  Інтеграція 2.1.3.  Безпека  даних  та  доступи 2.2.  Лабораторна  Інформаційна  Система  (ЛІС  "ЛабЕксперт") 2.2.1.  Функціональність 2.2.2.  Інтеграція  з  обладнанням 2.3.  Радіологічна  Інформаційна  Система  (РІС  "ДіагноРад") 2.3.1.  Особливості  РІС 2.3.2.  PACS- система 2.4.  Система  Управління  Чергами  та  Записами  (СУЧЗ  "Електронна  Реєстратура") 2.5.  Телемедична  Платформа  "Омега-Коннект"  3.  Медичне  Обладнання  та  Інфраструктура 3.1.  Відділення  Діагностики  та  Візуалізації 3.1.1.  Апарат  Магнітно-Резонансної  Томографії  (